## Import Packages - Libraries

In [1]:
from docplex.mp.model import Model
import time
import numpy as np
from os.path import join
import pickle
from datetime import datetime, timedelta
import pandas as pd

## Global Variable

In [2]:
input_path = join("io", "input")
output_path = join("io", "output")

## Load Input Data

In [3]:
def load_dict_from_file(filepath):
    """
    Load a dictionary from a pickle file.
    """
    with open(filepath, 'rb') as file:
        return pickle.load(file)

In [4]:
def load_data(output_path):
    schedules = load_dict_from_file(join(output_path, "fct_schedules_mean_sub"))
    gap_dict = load_dict_from_file(join(output_path, "fct_gap_passengers_sub"))
    stop_factor = load_dict_from_file(join(output_path, "interm_stop_factor"))
    return schedules, gap_dict, stop_factor

## Preprocess Parameters

In [5]:
def get_lines_and_stops(schedules):
    """
    Extracts and returns sorted lists of all distinct lines and all distinct stops.
    """
    bus_lines = sorted(schedules.keys())
    
    # Use a set to collect unique stops
    all_stops = set()
    for line in schedules:
        all_stops.update(schedules[line].keys())
    
    stops = sorted(all_stops)
    
    return bus_lines, stops

In [6]:
def get_schedules_actual_start_times(schedules):

    # Define reference start time
    start_time = datetime.strptime("06:59", "%H:%M")
    
    start_times_dict = {}
    
    for bus, stops in schedules.items():
        first_stop = next(iter(stops))  # Get the first stop key
        eta_minutes = stops[first_stop]
    
        # Convert to HH:MM time format
        eta_times = [(start_time + timedelta(minutes=eta)).strftime("%H:%M") for eta in eta_minutes]
    
        # Store in dict
        start_times_dict[bus] = eta_times
    return start_times_dict

In [7]:
def get_opt_parameters(output_path):
    schedules, gap_dict, stop_factor = load_data(output_path)

    bus_lines, stops = get_lines_and_stops(schedules)
    schedules_per_line = {line: max(len(times) for times in stops.values()) for line, stops in schedules.items()}
    max_gap_threshold = {stop_id: 200 for stop_id in stop_factor}
    
    start_times_dict = get_schedules_actual_start_times(schedules)
    
    max_lines = {'b00_w': 2, 'b02_w': 4, 'b05_w': 4, 'b06_w': 4, 'b07_w': 4, 'b10_w': 4, 'b12_w': 4,
                 'b14_w': 4, 'b30_w': 4, 'b31_w': 4, 'b33_w': 4, 'b37_w': 4, 'b39_w': 4}
    
    min_service_requirements = {
        '15085': {"min_service": 2, "time_frame": (0, 100)}
    }

    return schedules, gap_dict, stop_factor, bus_lines, stops, schedules_per_line, max_gap_threshold, start_times_dict, max_lines, min_service_requirements
    

## Set Up Optmimization Model

In [8]:
def create_opt_model():
    mdl = Model(name='bus_scheduling')
    return mdl

In [9]:
def define_decision_variable(mdl, schedules, schedules_per_line):
    x = mdl.binary_var_dict(
        keys=[(line, j) for line in schedules for j in range(1, schedules_per_line[line] + 1)],
        name='x')

    print ("Decision variable has been created")
    return x

## Setup Operational Constraints

In [10]:
def max_schedules_constraint(mdl, x, bus_lines, schedules_per_line, max_lines):
# Each bus line must run at least 1 schedule
    for line in bus_lines:
        if line != 'b00_w':
            constraint = mdl.add_constraint(mdl.sum(x[line, j] for j in range(1, schedules_per_line[line] + 1)) == max_lines[line], ctname=f'run_{line}')
            print(f"Constraint for {line}: {constraint}")
    return True

In [11]:
def start_end_limit_constraint(mdl, x, schedules_per_line):
    for j in range(1, schedules_per_line['b00_w'] + 1):  # Loop through all schedule indices
        mdl.add_constraint(x['b00_w', j] == 1, ctname=f'force_all_bus00_schedule_{j}')

    print('Start - End Limit Constraint has bee added successfully')
    return True

In [12]:
def minimum_stop_service_constraint(mdl, x, min_service_requirements, bus_lines, schedules, schedules_per_line):
    # Add constraints for minimum service at specific stops
    for stop, requirement in min_service_requirements.items():
        min_service = requirement["min_service"]
        time_frame = requirement["time_frame"]
        start_time, end_time = time_frame
    
        # Collect schedules that fall within the specified time frame
        valid_schedules = [
            x[bus, j]
            for bus in bus_lines  # Loop through all bus lines
            if stop in schedules[bus]  # Check if the bus serves this stop
            for j in range(1, schedules_per_line[bus] + 1)  # Loop through all schedules for the bus
            if schedules[bus][stop][j - 1] >= start_time and schedules[bus][stop][j - 1] <= end_time
        ]
        
        # Add the constraint for the minimum number of schedules
        mdl.add_constraint(
            sum(valid_schedules) >= min_service,
            f'min_service_{stop}'
        )

    print('Minimum Stop Service Constraint has been added')
    return True

## Setup Objective Function

In [13]:
def max_gap_constraint(mdl, x, stops, bus_lines, schedules, schedules_per_line, gap_dict, max_gap_threshold):
    # Define maximum gap variables for each stop
    max_gaps = {stop: mdl.continuous_var(name=f'max_gap_{stop}') for stop in stops}
    
    
    # Add constraints to define the maximum gaps based on the variable decisions
    gap_results = {}
    for stop in stops:
        print('Calculating Max Gap for stop:', stop)
        all_possible_times = []
        all_possible_times2 = []
        for line in bus_lines:           # Loop through each bus line
            if stop in schedules[line]:  # Check if the current stop is served by the current bus line
                for j in range(1, schedules_per_line[line] + 1):    # Loop through each schedule option (assuming there are 2 options per line)
                    time_expr = x[line, j] * schedules[line][stop][j-1]
                    all_possible_times.append(time_expr)     # Append the time to the list of possible times
                    all_possible_times2.append(schedules[line][stop][j-1])
        max_gap_var, is_lower_out_vars, is_greater_out_vars, is_totaly_out_vars, include_pairs_vars = calculate_max_gap(all_possible_times, all_possible_times2, max_gaps[stop], stop, mdl, gap_dict)
        gap_results[stop] = (max_gap_var, is_lower_out_vars, is_greater_out_vars, is_totaly_out_vars, include_pairs_vars)
    
        threshold = max_gap_threshold[stop]
        threshold_constraint = mdl.add_constraint(max_gap_var <= threshold, f'max_gap_threshold_{stop}')
        print(f"Threshold constraint for {stop}: {threshold_constraint}")
    
        gap_constraint = mdl.add_constraint(max_gaps[stop] == max_gap_var)
        print(f"Gap constraint for {stop}: {gap_constraint}")

    return max_gaps, gap_results



In [14]:
def calculate_max_gap(all_possible_times, all_possible_times2, max_gap_var, stop, mdl, gap_dict):
    print(f"Calculating gaps for stop {stop} with possible times: {all_possible_times}")
    # Example hardcoded limit
    gaps = []
    is_greater_out_vars = []  # List to store for debugging within the function scope
    is_lower_out_vars = []
    is_totaly_out_vars = []
    include_pairs_vars = []

    n_times = len(all_possible_times)    
    # Precompute min and max relationships
    min_matrix = [[None for _ in range(n_times)] for _ in range(n_times)]
    max_matrix = [[None for _ in range(n_times)] for _ in range(n_times)]

    for i in range(n_times - 1):
        for j in range(i + 1, n_times):
            min_matrix[i][j] = min(all_possible_times2[i], all_possible_times2[j])
            max_matrix[i][j] = max(all_possible_times2[i], all_possible_times2[j])


    # Here we loop over all possible time to calualte the max bettwen every pair
    for i in range(len(all_possible_times) - 1):
        for j in range(i + 1, len(all_possible_times)):
            gap_abs = mdl.continuous_var(name=f"gap_abs_{stop}_{i}_{j}", lb=0)
            gap = mdl.continuous_var(name=f"gap_{stop}_{i}_{j}", lb=0)
            gap_key = f"gap_{stop}_{i}_{j}_{all_possible_times2[i]}_{all_possible_times2[j]}"
            include_pairs = mdl.binary_var(name=f"include_pairs_{stop}_{i}_{j}")
            
            is_totally_out_ls = []
            #Here we check if that pair has no othe time between by looping again on the othe times
            for k in range(len(all_possible_times)):
                is_lower_out = mdl.binary_var(name=f"is_lower_out_{stop}_{i}_{j}_{k}")
                is_greater_out = mdl.binary_var(name=f"is_greater_out_{stop}_{i}_{j}_{k}")
                is_totaly_out = mdl.binary_var(name=f"combined_{stop}_{i}_{j}_{k}")
                if k != i and k != j:

                    min_ij = min_matrix[i][j]
                    max_ij = max_matrix[i][j]
                
                    # Indicator for is_lower_out
                    mdl.add_indicator(is_lower_out, all_possible_times[k] - min_ij >= 1, 0)
                    mdl.add_indicator(is_lower_out, all_possible_times[k] - min_ij <= 0, 1)
                    
                    # New indicator for is_greater_out
                    mdl.add_indicator(is_greater_out, max_ij - all_possible_times[k] >= 1, 0)
                    mdl.add_indicator(is_greater_out, max_ij - all_possible_times[k] <= 0, 1)

                    # If the other time is less than the minimum between time of the pair or max than the maximum 
                    # Then this pair has the potential to be inculded on the max time gap calculation 
                    mdl.add_constraint(is_totaly_out >= is_lower_out)
                    mdl.add_constraint(is_totaly_out >= is_greater_out)
                    mdl.add_constraint(is_totaly_out <= is_lower_out + is_greater_out)

                    is_lower_out_vars.append((is_lower_out, i, j, k))
                    is_greater_out_vars.append((is_greater_out, i, j, k))
                    is_totaly_out_vars.append((is_totaly_out, i, j, k))
                    is_totally_out_ls.append(is_totaly_out)        
            
            if gap_key in gap_dict:
                gap_value = gap_dict[gap_key]
            else:
                raise ValueError(f"Gap key {gap_key} not found in precomputed gaps")

            #Define binary variables for selection conditions
            is_selected_i = mdl.binary_var(name=f"is_selected_{stop}_{i}_{j}_{i}")
            is_selected_j = mdl.binary_var(name=f"is_selected_{stop}_{i}_{j}_{j}")
            
            # Add indicators to define when all_possible_times[i] and all_possible_times[j] are greater than 1
            # Constraints to determine if all_possible_times[i] and all_possible_times[j] are selected
            mdl.add_constraint(is_selected_i == (all_possible_times[i] >= 1))
            mdl.add_constraint(is_selected_j == (all_possible_times[j] >= 1))
            
            # Define an auxiliary binary variable to capture the AND condition
            z_and = mdl.binary_var(name=f"z_and_{stop}_{i}_{j}")
            
            # Add constraints for the AND condition
            mdl.add_constraint(z_and <= is_selected_i)
            mdl.add_constraint(z_and <= is_selected_j)
            mdl.add_constraint(z_and >= is_selected_i + is_selected_j - 1)

            mdl.add_constraint(include_pairs == mdl.min(is_totally_out_ls))
            
            mdl.add_indicator(include_pairs, gap == gap_value * z_and, 1)
            mdl.add_indicator(include_pairs, gap == 0, 0)
            
            gaps.append(gap)
            include_pairs_vars.append((include_pairs, i, j))

    if gaps:
        mdl.add_constraint(max_gap_var == mdl.max(gaps), f'max_gap_constraint_{stop}')

    return max_gap_var, is_lower_out_vars, is_greater_out_vars, is_totaly_out_vars, include_pairs_vars  # Return the maximum gap variable directly
    

In [15]:
def add_objective_function(mdl,stop_factor, max_gaps, stops):
    # Objective: Minimize the average maximum gap at each stop
    weighted_gaps = mdl.sum(stop_factor[stop] * max_gaps[stop] for stop in stops)
    objective = weighted_gaps / len(stops)
    mdl.minimize(objective)

    print('Objective Function has been created')
    return True

## Solve Model

In [16]:
def solve(mdl, x, gap_results, stops, bus_lines, start_times_dict, logs=False):
    # Solve the model
    solution = mdl.solve(log_output=True)

    if solution:
        final_schedule_df = print_solution(x, stops, gap_results, bus_lines, schedules_per_line, start_times_dict)
        print(f"Average maximum gap (Objective value): {mdl.objective_value}")
    else:
        print("No solution found")

    if logs:
        print_logs(stops, gap_results)

    return final_schedule_df

## Print Results & Logs

In [17]:
def print_solution(x, stops, gap_results, bus_lines, schedules_per_line, start_times_dict):
    bus_ls = []
    start_times_ls = []
    for stop in stops:
        max_gap_var, is_lower_out_vars, is_greater_out_vars, is_totaly_out_vars, include_pairs_vars = gap_results[stop]
        print(f"Maximum gap at {stop}: {max_gap_var.solution_value}")
    for line in bus_lines:
        if line == 'b00_w':
            continue
        print(f"Selected schedules for {line}:")     
        for j in range(1, schedules_per_line[line] + 1):
            print(f"  Schedule Starts At {start_times_dict[line][j-1]}: {x[line, j].solution_value}")
            if x[line, j].solution_value:
                bus_ls.append(line)
                start_times_ls.append(start_times_dict[line][j-1])
    final_schedule_df = pd.DataFrame(list(zip(bus_ls, start_times_ls)), columns=['bus_name', 'start_time'])
    return final_schedule_df    


In [18]:
def print_logs(stops, gap_results):
    for stop in stops:
        max_gap_var, is_lower_out_vars, is_greater_out_vars, is_totaly_out_vars, include_pairs_vars = gap_results[stop]
        print(f"is_lower_out values for {stop}:")
        for is_lower_out, i, j, k in is_lower_out_vars:
            print(f"is_lower_out for pair {i}-{j}-{k}: {is_lower_out.solution_value}")

        print(f"is_greater_out values for {stop}:")
        for is_greater_out, i, j, k in is_greater_out_vars:
            print(f"is_greater_out for pair {i}-{j}-{k}: {is_greater_out.solution_value}")

        print(f"is_totaly_out values for {stop}:")
        for is_totaly_out, i, j, k in is_totaly_out_vars:
            print(f"is_totaly_out for pair {i}-{j}-{k}: {is_totaly_out.solution_value}")
            
        print(f"include_pairs_vars values for {stop}:")
        for include_pairs, i, j in include_pairs_vars:
            print(f"include_pairs for pair {i}-{j}: {include_pairs.solution_value}")
        
    return True

## Optmization Pipeline

In [19]:
def optimize_bus_schedule(stops, schedules, schedules_per_line, bus_lines, max_lines, min_service_requirements, gap_dict, max_gap_threshold, stop_factor, start_times_dict):

    start_time = time.perf_counter()
    
    mdl = create_opt_model()
    x = define_decision_variable(mdl, schedules, schedules_per_line)
    status = max_schedules_constraint(mdl, x, bus_lines, schedules_per_line, max_lines)
    status = start_end_limit_constraint(mdl, x, schedules_per_line)
    status = minimum_stop_service_constraint(mdl, x, min_service_requirements, bus_lines, schedules, schedules_per_line)
    max_gaps, gap_results = max_gap_constraint(mdl, x, stops, bus_lines, schedules, schedules_per_line, gap_dict, max_gap_threshold)
    status = add_objective_function(mdl, stop_factor, max_gaps, stops)
    final_schedule_df = solve(mdl, x, gap_results, stops, bus_lines, start_times_dict, logs=False)

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(execution_time)

    return final_schedule_df

## Execution

In [ ]:
schedules, gap_dict, stop_factor, bus_lines, stops, schedules_per_line, max_gap_threshold, start_times_dict, max_lines, min_service_requirements = get_opt_parameters(output_path)

final_schedule_df = optimize_bus_schedule(stops, schedules, schedules_per_line, bus_lines, max_lines, min_service_requirements, gap_dict, max_gap_threshold, stop_factor, start_times_dict)


Decision variable has been created
Constraint for b02_w: run_b02_w: x_b02_w_1+x_b02_w_2+x_b02_w_3+x_b02_w_4+x_b02_w_5+x_b02_w_6+x_b02_w_7+x_b02_w_8 == 4
Constraint for b05_w: run_b05_w: x_b05_w_1+x_b05_w_2+x_b05_w_3+x_b05_w_4+x_b05_w_5+x_b05_w_6+x_b05_w_7+x_b05_w_8 == 4
Constraint for b06_w: run_b06_w: x_b06_w_1+x_b06_w_2+x_b06_w_3+x_b06_w_4+x_b06_w_5+x_b06_w_6+x_b06_w_7+x_b06_w_8 == 4
Constraint for b07_w: run_b07_w: x_b07_w_1+x_b07_w_2+x_b07_w_3+x_b07_w_4+x_b07_w_5+x_b07_w_6+x_b07_w_7+x_b07_w_8 == 4
Constraint for b10_w: run_b10_w: x_b10_w_1+x_b10_w_2+x_b10_w_3+x_b10_w_4+x_b10_w_5+x_b10_w_6+x_b10_w_7+x_b10_w_8 == 4
Constraint for b12_w: run_b12_w: x_b12_w_1+x_b12_w_2+x_b12_w_3+x_b12_w_4+x_b12_w_5+x_b12_w_6+x_b12_w_7+x_b12_w_8 == 4
Constraint for b14_w: run_b14_w: x_b14_w_1+x_b14_w_2+x_b14_w_3+x_b14_w_4+x_b14_w_5+x_b14_w_6+x_b14_w_7+x_b14_w_8 == 4
Constraint for b30_w: run_b30_w: x_b30_w_1+x_b30_w_2+x_b30_w_3+x_b30_w_4+x_b30_w_5+x_b30_w_6+x_b30_w_7+x_b30_w_8 == 4
Constraint for b31_w:

## Save Final Schedule

In [ ]:
final_schedule_df.to_csv(join(output_path, "fct_schedules.csv"), sep=',', encoding='utf-8', index=False)


In [ ]:
schedules, gap_dict, stop_factor, bus_lines, stops, schedules_per_line, max_gap_threshold, start_times_dict, max_lines, min_service_requirements = get_opt_parameters(output_path)
